In [1]:
from pymystem3 import Mystem
from deep_translator import GoogleTranslator
import inflect
import nltk
from langid import classify
import numpy as np

In [2]:
# словарь KEYWORDS_1 имеет вид "название особого токена" : "его 'перевод'"; тут лежат токены, 
# которые просто заменяются на свои питоновские аналоги
KEYWORDS_1 = {"функция": "def", "класс": "class", "вернуть": "return", "добавлять": "append", 
              "для": "for", "в": "in", "во": "in", "не": " not", "инициализация": "__init__", "если": "if", 
              "иначе": "else", "случайный": "random.random()", "объект": "self", "предел": "range", "от": "(", 
              "до": ",", "целый": "int"}
# словарь KEYWORDS_2 также имеет вид "название особого токена" : "его 'перевод'"; тут лежат токены, 
# которые не только заменяются на свои питоновские аналоги, но и требуют после себя наличие открытой скобки
KEYWORDS_2 = {"длина": "len", "минимум": "min", "максимум": "max"}
# словарь OPERATIONS имеет вид "название бинарной операции" : "её символ"
OPERATIONS = {"равно": "=", "равный": "=", "сумма": "+", "разность": "-", "поделить": "/", "больше": ">", 
              "мало": "<", "произведение": "*"}
# массив NO_WHITESPACE_AFTER содержит символы, после которых пробел не ставится
NO_WHITESPACE_AFTER = ["[", "("]
# массив NO_WHITESPACE_BEFORE содержит символы, перед которыми пробел не ставится
NO_WHITESPACE_BEFORE = ["(", "]", ")", ",", "[", ".append", ":"]

In [3]:
# функция parse_operation принимает на вход:
#
# stemmed_tokens: массив токенов после стемминга
#
# token_analyses: массив информации о токенах (если токены англоязычные, не текстовые или смешанные 
# (текстово-символьные), то на данном месте хранятся сами токены, если же токен состоит из русских букв, то на 
# данном месте хранится словарь из MyStem().analyze()); здесь и далее структура подобного массива одинакова
# 
# operation: строка-ключ бинарной операции из словаря OPERATIONS (см. выше)
#
# OPERATIONS: словарь с бинарными операциями (см. выше)
#
# что делает: заменяет в массиве токенов словесное описание операции на символ операции, избавляется от 
# вспомогательных слов вроде "на" в случае "произведение a на b"
#
# что возвращает: 
#
# модифицированный массив токенов
#
# модифицированный массив информации о токенах

def parse_operation(stemmed_tokens, token_analyses, operation, OPERATIONS):
    if operation == "равно" or operation == "равный" or operation == "больше" or operation == "мало":
        oper_ix = stemmed_tokens.index(operation)
        l1 = stemmed_tokens[:oper_ix]
        r1 = stemmed_tokens[oper_ix + 1:]
        l2 = token_analyses[:oper_ix]
        r2 = token_analyses[oper_ix + 1:]
        return l1 + [OPERATIONS[operation]] + r1, l2 + [OPERATIONS[operation]] + r2
    elif operation == "поделить":
        oper_ix = stemmed_tokens.index(operation)
        l1 = stemmed_tokens[:oper_ix]
        r1 = stemmed_tokens[oper_ix + 2:]
        l2 = token_analyses[:oper_ix]
        r2 = token_analyses[oper_ix + 2:]
        return l1 + [OPERATIONS[operation]] + r1, l2 + [OPERATIONS[operation]] + r2
    elif operation == "произведение":
        on_ix = stemmed_tokens.index("на")
        oper_ix = stemmed_tokens.index(operation)
        l1 = stemmed_tokens[1:oper_ix]
        s1 = stemmed_tokens[oper_ix + 1:on_ix]
        r1 = stemmed_tokens[on_ix + 1:]
        l2 = token_analyses[1:oper_ix]
        s2 = token_analyses[oper_ix + 1:on_ix]
        r2 = token_analyses[on_ix + 1:]
        return l1 + s1 + [OPERATIONS[operation]] + r1, l2 + s2 + [OPERATIONS[operation]] + r2 
    else:
        and_ix = stemmed_tokens.index("и")
        oper_ix = stemmed_tokens.index(operation)
        l1 = stemmed_tokens[1:oper_ix]
        s1 = stemmed_tokens[oper_ix + 1:and_ix]
        r1 = stemmed_tokens[and_ix + 1:]
        l2 = token_analyses[1:oper_ix]
        s2 = token_analyses[oper_ix + 1:and_ix]
        r2 = token_analyses[and_ix + 1:]
        return l1 + s1 + [OPERATIONS[operation]] + r1, l2 + s2 + [OPERATIONS[operation]] + r2 

In [4]:
# функция parse_script принимает на вход:
#
# filename: имя входного файла с русским текстом программы
#
# что делает: генерирует на основе входного файла выходной файл output.py
#
# что возвращает: 
# ---

def parse_script(filename):
    f = open(filename, "r")
    g = open("output.py", "w")
    classes = {}
    class_methods = {}
    for line in f:
        if len(line.strip()) == 0:
            g.write("\n")
        else:
            preparsed_string, token_info = preparse_string(line)
            parsed_string, token_info, classes, class_methods = collect_functions_and_classes(preparsed_string, 
                                                                                              token_info, classes, 
                                                                                              class_methods)
            parsed_string = preparse_variables(parsed_string, token_info, classes, class_methods)
            final_string = handle_whitespace(parsed_string, NO_WHITESPACE_AFTER, NO_WHITESPACE_BEFORE)
            g.write(final_string + "\n")
    f.close()
    g.close()

In [5]:
# функция preparse_string принимает на вход:
#
# string: строка
#
# что делает: разбивает строку на токены, к которым применяется стемминг; получает информацию о токенах; 
# обрабатывает словесное описание бинарных операций на основе словаря OPERATION, ключевых слов на основе словарей
# KEYWORDS_1, KEYWORDS_2; вырезает вспомогательные слова; расставляет скобки
#
# что возвращает: 
#
# preparsed_string: массив токенов
# token_info: массив информации о токенах

def preparse_string(string):
    tokens_to_be_ignored = ["число", "к", "из"]
    brace_balance = 0
    i = 0
    preparsed_string = []
    while i < len(list(string)) and (list(string)[i] == " " or list(string)[i] == "\t"):
        if len(preparsed_string) > 0:
            preparsed_string[0] += list(string)[i]
        else:
            preparsed_string.append(list(string)[i])
        string = string[1:]
    tokens = nltk.word_tokenize(string)
    mystem = Mystem()
    stemmed_tokens = [mystem.lemmatize(token)[0] if token.isalpha() else token for token in tokens]
    token_analyses = [mystem.analyze(token)[0]['analysis'] if 'analysis' in mystem.analyze(token)[0].keys() 
                      and classify(token)[0] != "en" else token for token in tokens]
    token_analyses = [summary[0] if type(summary) == list and len(summary) > 0 else summary 
                      for summary in token_analyses]
    operation_flag = False
    for operation in OPERATIONS.keys():
        if operation in stemmed_tokens:
            if not operation_flag:
                stemmed_tokens, token_analyses = parse_operation(preparsed_string + stemmed_tokens, 
                                                                 preparsed_string + token_analyses, 
                                                                 operation, OPERATIONS)
                operation_flag = True
            else:
                stemmed_tokens, token_analyses = parse_operation(stemmed_tokens, token_analyses, 
                                                                 operation, OPERATIONS)
    token_info = []
    for i in range(len(stemmed_tokens)):
        token = stemmed_tokens[i]
        if token in KEYWORDS_1.keys():
            preparsed_string.append(KEYWORDS_1[token])
            token_info.append(KEYWORDS_1[token])
            if token == "от":
                brace_balance += 1
        elif token in KEYWORDS_2.keys():
            preparsed_string.append(KEYWORDS_2[token])
            preparsed_string.append("(")
            token_info.append(KEYWORDS_2[token])
            token_info.append("(")
            brace_balance += 1
        elif token in [" "*n for n in range(1, 20)] or token in tokens_to_be_ignored:
            continue
        else:
            preparsed_string.append(token)
            token_info.append(token_analyses[i])
    if brace_balance > 0:
        if preparsed_string[-1] == ":":
            preparsed_string.insert(-1, ")")
            token_info.insert(-1, ")")
        else:
            preparsed_string.append(")")
            token_info.append(")")
        if brace_balance > 1:
            for i in range(len(preparsed_string) - 2, 0, -1):
                if preparsed_string[i].isalpha():
                    preparsed_string.insert(i + 1, ")")
                    token_info.insert(i + 1, ")")
                    break
    return preparsed_string, token_info

In [6]:
# функция translate_function_and_class принимает на вход:
#
# name_list: массив токенов для перевода - название функции, класса или метода класса
#
# t_info: массив информации о токенах
#
# что делает: с помощью гугл-переводчика переводит название функции, класса или метода класса; здесь
# рассматривается только несколько случаев: если в массиве больше трёх элементов или только один элемент, то
# перевод пословный, если в массиве два элемента, то рассматривается случай двух существительных (второе 
# существительное выступает в качестве определения первого, при переводе они меняются местами)
#
# что возвращает: 
#
# строку-имя функции, класса или метода класса, при необходимости склеенное с помощью нижних подчёркиваний

def translate_function_and_class(name_list, t_info):
    pt = GoogleTranslator(source="russian", target="english")
    translated = []
    if len(name_list) >= 3 or len(name_list) == 1:
        for word in name_list:
            translated.append(pt.translate(word))
        return "_".join(translated)
    else:
        morphology = [t["gr"] for t in t_info]
        if "S" in morphology[0] and "S" in morphology[1]:
            # сущ (ед), сущ (ед) / сущ (мн) -> бутылка воды / список людей
            if "ед" in morphology[0]:
                return "_".join([pt.translate(name_list[1]), pt.translate(name_list[0])])
        else:
            return "_".join(name_list)

In [7]:
# функция collect_functions_and_classes принимает на вход:
#
# preparsed_string: массив токенов
#
# t_info: массив информации о токенах
#
# classes: словарь вида "название класса": "перевод названия класса"
#
# class_methods: словарь вида "название метода класса": "перевод названия метода класса"
#
# что делает: в строках-инициализациях функций, классов и методов классов находит и перевод с помощью функции
# translate_function_and_class их названия; при необходимости дополняет словари classes и class_methods (изначально
# они пустые)
#
# что возвращает: 
#
# parsed_string: массив токенов
#
# token_info: массив информации о токенах
#
# classes: словарь вида "название класса": "перевод названия класса"
#
# class_methods: словарь вида "название метода класса": "перевод названия метода класса"

def collect_functions_and_classes(preparsed_string, t_info, classes, class_methods):
    parsed_string = []
    token_info = []
    if preparsed_string[0] == "def":
        parsed_string.append("def")
        token_info.append("def")
        function_name = []
        i = 1
        while i < len(preparsed_string) and preparsed_string[i] != "(":
            function_name.append(preparsed_string[i])
            i += 1
        translated = translate_function_and_class(function_name, t_info[1:i])
        parsed_string.append(translated) 
        parsed_string.append("(")
        parsed_string += preparsed_string[i + 1:]
        token_info.append(translated)
        token_info.append("(")
        token_info += t_info[i + 1:]
    elif preparsed_string[0] in [" "*n for n in range(1, 20)] and preparsed_string[1] == "def":
        parsed_string.append(preparsed_string[0])
        parsed_string.append(preparsed_string[1])
        token_info.append(preparsed_string[0])
        token_info.append(preparsed_string[1])
        class_function_name = []
        i = 2
        while i < len(preparsed_string) and preparsed_string[i] != "(":
            class_function_name.append(preparsed_string[i])
            i += 1
        class_function_name_one_word = "_".join(class_function_name)
        if class_function_name_one_word not in class_methods.keys():
            class_methods[class_function_name_one_word] = {}
        translated = translate_function_and_class(class_function_name, t_info[1:i - 1])
        class_methods[class_function_name_one_word] = translated
        parsed_string.append(translated)
        parsed_string.append("(")
        parsed_string += preparsed_string[i + 1:]
        token_info.append(translated)
        token_info.append("(")
        token_info += t_info[i:]
    elif preparsed_string[0] == "class":
        parsed_string.append("class")
        token_info.append("class")
        class_name = []
        i = 1
        while i < len(preparsed_string) and preparsed_string[i] != ":":
            class_name.append(preparsed_string[i])
            i += 1
        class_name_one_word = "_".join(class_name)
        if class_name_one_word not in classes.keys():
            classes[class_name_one_word] = {}
        translated = translate_function_and_class(class_name, t_info[1:i])
        translated = translated[0].upper() + translated[1:]
        classes[class_name_one_word] = translated
        parsed_string.append(translated)
        parsed_string.append(":")
        token_info.append(translated)
        token_info.append(":")
    else:
        parsed_string = preparsed_string
        if preparsed_string[0] in [" "*n for n in range(1, 20)]:
            token_info = [preparsed_string[0]] + t_info
        else:
            token_info = t_info
    return parsed_string, token_info, classes, class_methods

In [8]:
# функция handle_list принимает на вход:
#
# parsed_string: массив токенов
#
# t_info: массив информации о токенах
#
# что делает: если в строке нет информации о работе с элементами массивов, то не делает ничего; иначе обрабатывает
# добавление элемента в массив (в т.ч. с помощью информации о падежах слов) и вызов элемента массива
#
# что возвращает: 
#
# parsed_string: массив токенов
#
# t_info: массив информации о токенах

def handle_list(parsed_string, t_info):
    if "элемент" in parsed_string: # a[i]
        if "append" not in parsed_string: # array.append(a[i])
            ix = parsed_string.index("элемент")
            elem = parsed_string[ix - 3]
            for i in range(1, len(parsed_string) - ix):
                i0 = i
                if not parsed_string[ix + i].isalpha():
                    break
            list_name_ixs = [ix + i for i in range(1, i0 + 1)]
            list_name = [parsed_string[ix + i] for i in range(1, i0 + 1)]
            if not list_name[-1].isalpha():
                list_name_ixs = list_name_ixs[:-1]
                list_name = list_name[:-1]
            preparsed_string = parsed_string[:ix - 4] + list_name + ["[", elem, "]"] + \
                               parsed_string[max(list_name_ixs) + 1:]
            token_info = t_info[:ix - 4] + list(np.array(t_info)[list_name_ixs]) + ["[", elem, "]"] + \
                         t_info[max(list_name_ixs) + 1:]
            return preparsed_string, token_info
        else: # a[i] = ...
            ix = parsed_string.index("элемент")
            append_ix = parsed_string.index("append")
            elem = parsed_string[ix - 3]
            first_list_name_ixs = [append_ix + i for i in range(1, ix - 4) 
                                   if parsed_string[append_ix + i].isalpha()]
            first_list_name = [parsed_string[append_ix + i] for i in range(1, ix - 4) 
                               if parsed_string[append_ix + i].isalpha()]
            second_list_name_ixs = [ix + i for i in range(1, len(parsed_string) - ix) 
                                    if parsed_string[ix + i].isalpha()]
            second_list_name = [parsed_string[ix + i] for i in range(1, len(parsed_string) - ix) 
                                if parsed_string[ix + i].isalpha()]
            preparsed_string = parsed_string[:append_ix] + first_list_name + [".append", "("] + \
                               second_list_name + ["[", elem, "]", ")"]
            token_info = t_info[:append_ix] + list(np.array(t_info)[first_list_name_ixs]) + \
                         [".append", "("] + list(np.array(t_info)[second_list_name_ixs]) + ["[", elem, "]", ")"]
            return preparsed_string, token_info
    elif "append" in parsed_string and "элемент" not in parsed_string: # array.append(element)
        append_ix = parsed_string.index("append")
        elem_name_ixs = [append_ix + i for i in range(1, len(parsed_string) - append_ix) 
                         if parsed_string[append_ix + i].isalpha() and "вин,ед" in t_info[append_ix + i]["gr"]]
        elem_name = [parsed_string[append_ix + i] for i in range(1, len(parsed_string) - append_ix) 
                         if parsed_string[append_ix + i].isalpha() and "вин,ед" in t_info[append_ix + i]["gr"]]
        preparsed_string = parsed_string[:append_ix] + parsed_string[append_ix + 1:min(elem_name_ixs)] + \
                           [".append", "("] + elem_name + [")"]
        token_info = t_info[:append_ix] + t_info[append_ix + 1:min(elem_name_ixs)] + [".append", "("] + \
                     list(np.array(t_info)[elem_name_ixs]) + [")"]
        return preparsed_string, token_info
    else:
        return parsed_string, t_info

In [9]:
# функция handle_classes принимает на вход:
#
# var_name: массив токенов-имя переменной, функции и т.д.
#
# t_info: массив информации о токенах
#
# classes: словарь вида "название класса": "перевод названия класса"
#
# class_methods: словарь вида "название метода класса": "перевод названия метода класса"
#
# что делает: обрабатывает имена классов и методов классов в строках, где нет их инициализации; для примера, из 
# массива ['первая', 'координата', 'город'] делает массив ['город.первая', 'координата']
#
# что возвращает: 
#
# var_name: новый массив токенов-имя переменной, функции и т.д.
#
# t_info: новый массив информации о токенах
#
# class_flag: флаг, который равен True в случае если работа производилась с именем класса (чтобы потом написать его
# название с большой буквы), False иначе

def handle_classes(var_name, t_info, classes, class_methods):
    class_flag = False
    token_info = []
    if len(var_name) > 2 and var_name[-1] in classes.keys():
        end = var_name[-1] + "." + var_name[0]
        end_token = str(t_info[-1]) + '.' + str(t_info[0])
        token_info = [end_token] + t_info[1:-1]
        return [end] + var_name[1:-1], token_info, class_flag
    elif len(var_name) == 1 and var_name[0] in classes.keys():
        class_flag = True
        return var_name, t_info, class_flag
        
    elif len(var_name) > 1 and var_name[0] in class_methods.keys() and var_name[1] in classes.keys():
        end = var_name[-1] + "." + var_name[0]
        end_token = str(t_info[-1]) + "." + str(t_info[0])
        token_info = t_info[1:-1] + [end_token]
        return var_name[1:-1] + [end], token_info, class_flag
    else:
        return var_name, t_info, class_flag

In [10]:
# функция translate_two_words принимает на вход:
#
# name_list: массив токенов из двух элементов для перевода - название переменной, функции и т.д.
#
# t_info: массив информации о токенах
#
# что делает: с помощью гугл-переводчика переводит название переменной, функции и т.д.; здесь рассматривается
# несколько случаев: случай двух существительных (второе существительное выступает в качестве определения первого, 
# при переводе они меняются местами), случай прилагательного или порядкового числительного и существительного (при
# переводе порядок прямой); решается проблема, когда при переводе из одного слова возникает несколько
#
# что возвращает: 
#
# строку-имя переменной, функции и т.д., при необходимости склеенное с помощью нижних подчёркиваний

def translate_two_words(name_list, t_info):
    pt = GoogleTranslator(source="russian", target="english")
    morphology = [t["gr"] for t in t_info]
    if "S" in morphology[0] and "S" in morphology[1]:
        # сущ (ед), сущ (ед) / сущ (мн) -> бутылка воды, список людей
        if "ед" in morphology[0]:
            l = pt.translate(name_list[1])
            if "the" in l:
                l = l[4:]
            r = pt.translate(name_list[0])
            if "the" in r:
                r = r[4:]
            return "_".join([l, r])
        # сущ (мн), сущ (ед) / сущ (мн) -> бутылки воды, списки людей
        elif "мн" in morphology[0]:
            l = pt.translate(name_list[1])
            if "the" in l:
                l = l[4:]
            r = pt.translate(name_list[0])
            if "the" in r:
                r = r[4:]
            engine = inflect.engine()
            return "_".join([l, engine.plural(r)])
        else:
            return "_".join(name_list)
    if "S" in morphology[1] and ("A" in morphology[0] or "ANUM" in morphology[0]):
        # прил, сущ / порядк числ, сущ -> новый дом, первый квартал
        l = pt.translate(name_list[0])
        if "the" in l:
            l = l[4:]
        r = pt.translate(name_list[1])
        if "the" in r:
            r = r[4:]
        return "_".join([l, r])
    else:
        return "_".join(name_list)

In [11]:
# функция translate_variables принимает на вход:
#
# name_list: массив токенов для перевода - название переменной, функции и т.д.
#
# t_info: массив информации о токенах
#
# что делает: с помощью гугл-переводчика переводит название переменной, функции и т.д.; здесь рассматривается
# несколько случаев: если в массиве больше трёх элементов или только один элемент, то перевод пословный, если в 
# массиве два элемента, то применяется функция translate_two_words; особым образом обрабатываются классы и их 
# методы
#
# что возвращает: 
#
# строку-имя переменной, функции и т.д., при необходимости склеенное с помощью нижних подчёркиваний

def translate_variables(name_list, t_info):
    pt = GoogleTranslator(source="russian", target="english")
    translated = []
    if len(name_list) >= 3 or len(name_list) == 1:
        for word in name_list:
            tr = pt.translate(word)
            if "the" not in tr:
                translated.append(tr)
            else:
                translated.append(tr[4:])
        return "_".join(translated)
    else:
        if "." in name_list[0]:
            point_ix = name_list[0].index(".")
            point_ix_token = t_info[0].index(".")
            new_t_info = [eval(t_info[0][point_ix_token + 1:])] + [t_info[1]]
            right = translate_two_words([name_list[0][point_ix + 1:]] + [name_list[1]], new_t_info)
            left = pt.translate(name_list[0][:point_ix])
            if "the" not in left:
                return left + "." + right
            else:
                return left[4:] + "." + right
        elif "." in name_list[1]:
            point_ix = name_list[1].index(".")
            point_ix_token = t_info[1].index(".")
            new_t_info = [t_info[0]] + [eval(t_info[1][:point_ix_token])]
            left = translate_two_words([name_list[0]] + [name_list[1][:point_ix]], new_t_info)
            right = pt.translate(name_list[1][point_ix + 1:])
            if "the" not in right:
                return left + "." + right
            else:
                return left + "." + right[4:]
        else:
            return translate_two_words(name_list, t_info)

In [12]:
# функция preparse_variables принимает на вход:
#
# parsed_string: массив токенов
#
# t_info: массив информации о токенах
#
# classes: словарь вида "название класса": "перевод названия класса"
#
# class_methods: словарь вида "название метода класса": "перевод названия метода класса"
#
# что делает: находит в строке название переменной, функции и т.д., переводит его с помощью функции 
# translate_variables; особым образом обрабатываются названия с self
#
# что возвращает: 
#
# preparsed_string: массив токенов

def preparse_variables(parsed_string, t_info, classes, class_methods):
    preparsed_string = []
    token_info = []
    var_name = []
    var_ixs = []
    p_string, tk_info = handle_list(parsed_string, t_info)
    for i in range(len(p_string)):
        if p_string[i].isalpha() and classify(p_string[i])[0] != "en":
            var_name.append(p_string[i])
            var_ixs.append(i)
        elif p_string[i] == "self":
            var_name.append(p_string[i])
        else:
            if len(var_name) > 0:
                if "self" not in var_name:
                    tt_info = list(np.array(tk_info)[var_ixs])
                    new_var_name, var_info, class_flag = handle_classes(var_name, tt_info, classes, class_methods)
                    translated = translate_variables(new_var_name, var_info)
                    if not class_flag:
                        preparsed_string.append(translated)
                        preparsed_string.append(p_string[i])
                        var_name = []
                        var_ixs = []
                    else:
                        preparsed_string.append(translated[0].upper() + translated[1:])
                        preparsed_string.append(p_string[i])
                        var_name = []
                        var_ixs = []
                elif len(var_name) > 1:
                    tt_info = list(np.array(tk_info)[var_ixs])
                    new_var_name, var_info, class_flag = handle_classes(var_name[:-1], tt_info, classes, 
                                                                        class_methods)
                    translated = translate_variables(new_var_name, var_info)
                    preparsed_string.append("self." + translated)
                    preparsed_string.append(p_string[i])
                    var_name = []
                    var_ixs = []
                else:
                    preparsed_string.append("self")
                    preparsed_string.append(p_string[i])
                    var_name = []
                    var_ixs = []
            else:
                preparsed_string.append(p_string[i])
    if len(var_name) > 0:
        if "self" not in var_name:
            tt_info = list(np.array(tk_info)[var_ixs])
            new_var_name, var_info, class_flag = handle_classes(var_name, tt_info, classes, class_methods)
            translated = translate_variables(new_var_name, var_info)
            preparsed_string.append(translated)
            var_name = []
            var_ixs = []
        elif len(var_name) > 1:
            tt_info = list(np.array(tk_info)[var_ixs])
            new_var_name, var_info, class_flag = handle_classes(var_name[:-1], tt_info, classes, class_methods)
            translated = translate_variables(new_var_name, var_info)
            preparsed_string.append("self." + translated)
            var_name = []
            var_ixs = []
        else:
            preparsed_string.append("self")
            var_name = []
            var_ixs = []
    return preparsed_string

In [13]:
# функция handle_whitespace принимает на вход:
#
# string: массив токенов
#
# NO_WHITESPACE_AFTER: см. выше
#
# NO_WHITESPACE_BEFORE: см. выше
#
# что делает: правильно расставляет пробелы между токенами
#
# что возвращает: 
#
# финальную строку

def handle_whitespace(string, NO_WHITESPACE_AFTER, NO_WHITESPACE_BEFORE):
    final_string = [string[0]]
    for i in range(1, len(string)):
        if string[i] in NO_WHITESPACE_BEFORE:
            if string[i - 1] in NO_WHITESPACE_AFTER:
                final_string += [string[i]]
            else:
                if string[i] == "[" and string[i - 1] == "=":
                    final_string += [" ", string[i]]
                else:
                    final_string += [string[i]]
        elif string[i] not in NO_WHITESPACE_BEFORE:
            if string[i - 1] in NO_WHITESPACE_AFTER:
                final_string += [string[i]]
            else:
                final_string += [" ", string[i]]
    return "".join(final_string)

In [14]:
# запускаем
parse_script("genetic_algorithm.txt")

In [15]:
# проверяем, сгенерированный скрипт запускается
! python output.py